In [1]:
import pandas as pd
import numpy as np

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns

In [10]:
target = np.asanyarray(concrete_data[['Strength']])
concrete_data_columns = concrete_data.columns #
predictors = np.asanyarray(concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] )

In [12]:
from sklearn.preprocessing import StandardScaler
scaler_predictors = StandardScaler()
scaler_target = StandardScaler()
scaled_predictors = scaler_predictors.fit_transform(predictors)
scaled_target = scaler_target.fit_transform(target)

In [14]:
scaled_predictors[0:5]

array([[ 2.47791487, -0.85688789, -0.84714393, -0.91676439, -0.62044832,
         0.86315424, -1.21767004, -0.27973311],
       [ 2.47791487, -0.85688789, -0.84714393, -0.91676439, -0.62044832,
         1.05616419, -1.21767004, -0.27973311],
       [ 0.49142531,  0.79552649, -0.84714393,  2.17546125, -1.03914281,
        -0.52651741, -2.24091709,  3.55306569],
       [ 0.49142531,  0.79552649, -0.84714393,  2.17546125, -1.03914281,
        -0.52651741, -2.24091709,  5.05767679],
       [-0.79045879,  0.678408  , -0.84714393,  0.48879272, -1.03914281,
         0.07052671,  0.64788402,  4.97848673]])

In [15]:
scaled_target[0:5]

array([[2.64540763],
       [1.56142148],
       [0.26662698],
       [0.3133402 ],
       [0.50797859]])

Let's save the number of predictors to ncols_ since we will need this number when building our network.

In [13]:
n_cols = scaled_predictors.shape[1] 

import keras library 
you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras
if you import like this you wont recieve any further error

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.optimizers import Adam

Train and Test data by using train_test_split

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_predictors, test_predictors, train_target, test_target = train_test_split(scaled_predictors, scaled_target, test_size=0.3, random_state=3)

In [20]:
print(train_predictors.shape,train_target.shape)
print(test_predictors.shape,test_target.shape)

(721, 8) (721, 1)
(309, 8) (309, 1)


Use the Keras library to build a neural network with the following:

 One hidden layer of 10 nodes, and a ReLU activation function
 Use the adam optimizer and the mean squared error  as the loss function

In [21]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [22]:
model = regression_model()

In [23]:
model.fit(train_predictors, train_target, validation_data=(test_predictors, test_target), epochs=50, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 - 2s - loss: 1.2755 - val_loss: 1.1856
Epoch 2/50
721/721 - 0s - loss: 1.0830 - val_loss: 1.0346
Epoch 3/50
721/721 - 0s - loss: 0.9330 - val_loss: 0.9161
Epoch 4/50
721/721 - 0s - loss: 0.8196 - val_loss: 0.8187
Epoch 5/50
721/721 - 0s - loss: 0.7283 - val_loss: 0.7446
Epoch 6/50
721/721 - 0s - loss: 0.6567 - val_loss: 0.6864
Epoch 7/50
721/721 - 0s - loss: 0.5992 - val_loss: 0.6390
Epoch 8/50
721/721 - 0s - loss: 0.5536 - val_loss: 0.6011
Epoch 9/50
721/721 - 0s - loss: 0.5172 - val_loss: 0.5666
Epoch 10/50
721/721 - 0s - loss: 0.4861 - val_loss: 0.5417
Epoch 11/50
721/721 - 0s - loss: 0.4588 - val_loss: 0.5179
Epoch 12/50
721/721 - 0s - loss: 0.4373 - val_loss: 0.4956
Epoch 13/50
721/721 - 0s - loss: 0.4182 - val_loss: 0.4787
Epoch 14/50
721/721 - 0s - loss: 0.4013 - val_loss: 0.4632
Epoch 15/50
721/721 - 0s - loss: 0.3874 - val_loss: 0.4497
Epoch 16/50
721/721 - 0s - loss: 0.3744 - val_loss: 0.4373
Epoch 17/50
721/721

In [24]:
yhat = model.predict(test_predictors)

In [26]:
yhat[0:5]

array([[-0.8710821 ],
       [ 0.26539585],
       [ 0.27025312],
       [-0.2740717 ],
       [ 1.8145323 ]], dtype=float32)

ERROR! Session/line number was not unique in database. History logging moved to new session 466


In [28]:
test_target[0:5]

array([[-1.06949074],
       [ 0.37861892],
       [ 0.83556999],
       [-0.47180115],
       [ 2.6040844 ]])

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [31]:
print(mean_squared_error(yhat,test_target))
print(mean_absolute_error(yhat,test_target))
print(r2_score(yhat,test_target))

0.2625884383748285
0.3854327156370485
0.6061348822650865
